
Dataset : https://www.kaggle.com/datasnaek/chess<br/><br/>


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

lichess = pd.read_csv('../input/chess/games.csv')
print(lichess.shape)
lichess.head()

### Manipulation de données

In [ ]:
# Afficher les 1er, 2ème et 3ème quartiles pour les variables 'turns', 'white_rating' et 'black_rating'
lichess[['turns', 'white_rating', 'black_rating']].describe().loc[['25%', '50%', '75%']]

In [ ]:
# Afficher les 1er, 2ème et 3ème quartiles pour les variables 'turns' et 'rating' (contenant l'ensemble des valeurs des variables 'white_rating' et 'black_rating')
# lichess[['turns']].describe().loc[['25%', '50%', '75%']]
wr = lichess['white_rating'].to_frame().rename(columns={"white_rating": "rating"})
br = lichess['black_rating'].to_frame().rename(columns={"black_rating": "rating"})
lichess[['turns']].describe().loc[['25%', '50%', '75%']].\
    merge(pd.concat([wr, br]).describe().loc[['25%', '50%', '75%']], left_index = True, right_index = True)

In [ ]:
# Afficher le nombre de parties par ouverture ('opening_eco')
lichess.opening_eco.value_counts()

In [ ]:
#Afficher 5 ouvertures les plus populares ('opening_eco')
lichess.opening_eco.value_counts().sort_values(ascending = False).head(5)

In [ ]:
#Afficher le nombre moyen de tours par ouverture ('opening_eco')
lichess.groupby('opening_eco').turns.mean()

In [ ]:
#Afficher 5 joueurs qui ont joué le plus de parties en considérant uniquement ceux qui ont joué avec les blancs
lichess.white_id.value_counts().sort_values(ascending = False).head(5)

In [ ]:
#Afficher 5 joueurs qui ont joué le plus de parties (avec les blancs ou avec les noirs)
players = pd.merge(lichess.white_id.value_counts(), lichess.black_id.value_counts(), how='outer', left_index=True, right_index=True).\
    fillna(0)
players = players.assign(total_games = players.white_id + players.black_id)[['total_games']]
players.total_games.sort_values(ascending = False).head(5)

In [ ]:
# Pour chaque ouverture ('opening_eco') afficher le nombre de parties jouées et 
# le pourcentage des victoires des blancs (sur le nombre total des parties pour une ouverture donnée)
res = lichess.groupby('opening_eco').agg(total_games=('id', 'size'), ratio_white_winner=('winner', lambda x: np.mean(x == 'white')))
res.head()

In [ ]:
# Afficher uniquement les ouvertures avec plus de 100 parties jouées et 
# les trier dans l'ordre décroissant du pourcentage des victoires des blancs
res = lichess.groupby('opening_eco').\
    agg(total_games=('id', 'size'), ratio_white_winner=('winner', lambda x: np.mean(x == 'white'))).\
    query('total_games > 100').\
    sort_values('ratio_white_winner', ascending=False)
res.head()

In [ ]:
# Formuler et implémenter votre propre requête comprenant une sélection des lignes sur une condition 
# basant sur les valeurs de 2 variables et une agrégation de données par groupes de votre choix.
lichess.loc[(lichess['victory_status'] == 'mate') & (lichess['rated']), ['white_rating','black_rating']].agg([min, np.mean, max])

### Visualisation de données

In [ ]:
# Afficher un histogramme représentant la répartition du nombre de tours
sns.displot(x='turns', data=lichess, kind='hist', aspect=2.5)

In [ ]:
# Afficher un histogramme représentant la répartition du nombre de tours par classe d'ouverture (A, B, C, ...)
sns.displot(x='turns', data=lichess, hue=[x[0] for x in lichess.opening_eco], aspect=2.5)

In [ ]:
# Afficher un histogramme représentant la répartition du nombre de tours par classe d'ouverture (A, B, C, ...)
lichess['opening_class'] = [x[0] for x in lichess.opening_eco]
g = sns.FacetGrid(lichess, col='opening_class')
p = g.map(sns.histplot, 'turns')

In [ ]:
# Afficher un nuage de points représentant la relation entre le score des joueurs jouant les uns contre les autres. 
# Différencier les points par les valeurs de la variable 'rated'
sns.relplot(x='white_rating', y='black_rating', hue='rated', kind='scatter', data=lichess)

In [ ]:
# Prendre en compte uniquement des parties de 10 minutes par joueur ('increment_code': 10+..) 
# qui étaient commencées par "d4".
sns.relplot(x='white_rating', y='black_rating', hue='rated', kind='scatter', 
            data=lichess[(lichess['increment_code'].str.startswith('10+')) & (lichess['moves'].str.startswith('d4 '))])

In [ ]:
# Afficher la distribution (box plot) du nombre de tours ('turns') par ouverture 
# pour 2 ouvertures de votre choix ('opening_eco')
data = lichess[lichess.opening_eco.isin(['A00', 'B00'])] 
sns.catplot(x='opening_eco', y='turns', data=data, kind='box', height=6, aspect=1.3)

In [ ]:
# Afficher la distribution (box plot) du nombre de tours ('turns') par ouverture 
# pour 5 ouvertures les plus populaires ('opening_eco')
openings_top = lichess.opening_eco.value_counts().sort_values(ascending = False).head(5)
data = lichess[lichess.opening_eco.isin(openings_top.index)] 
sns.catplot(x='opening_eco', y='turns', data=data, kind='box', height=6, aspect=1.3)

In [ ]:
import datetime
lichess['rating_dt'] = [datetime.datetime.fromtimestamp(x / 1000) for x in lichess['last_move_at']]

In [ ]:
# Afficher l'évolution du score dans le temps pour un joueur de votre choix
# L'axe du temps doit être construit à partir de la variable `last_move_at` qui doit être transformée : 
# ses valeurs correspondent au nombre de millisecondes depuis le 01/01/1970 00:00:00 GMT (cf librairie datetime).
wr = lichess.loc[lichess['white_id'] == "a_p_t_e_m_u_u", ['rating_dt', 'white_rating']].rename(columns={"white_rating": "rating"})
br = lichess.loc[lichess['black_id'] == "a_p_t_e_m_u_u", ['rating_dt', 'black_rating']].rename(columns={"black_rating": "rating"})
data = pd.concat([wr, br])
sns.relplot(x = 'rating_dt', y = 'rating', data = data, kind = 'line', height = 5, aspect = 4)

In [ ]:
# Afficher l'évolution du score dans le temps pour le joueur qui a joué le plus de parties 
# (peu importe avec les blancs ou avec les noirs)
players = pd.merge(lichess.white_id.value_counts(), lichess.black_id.value_counts(), how='outer', left_index=True, right_index=True).fillna(0)
players = players.assign(total_games = players.white_id + players.black_id)[['total_games']]
p = players.total_games.sort_values(ascending = False).index[0]
wr = lichess.loc[lichess['white_id'] == p, ['rating_dt', 'white_rating']].rename(columns={"white_rating": "rating"})
br = lichess.loc[lichess['black_id'] == p, ['rating_dt', 'black_rating']].rename(columns={"black_rating": "rating"})
data = pd.concat([wr, br])
sns.relplot(x = 'rating_dt', y = 'rating', data = data, kind = 'line', height = 5, aspect = 4)